# Scraping Google Maps with Python

[Web scraping](https://en.wikipedia.org/wiki/Web_scraping) is the process of extracting data from web pages using software. There are many [techniques](https://en.wikipedia.org/wiki/Web_scraping#Techniques) to scrape data: computer vision, manual copy and pasting, pattern matching, etc – for this tutorial, we will use [HTML parsing](https://en.wikipedia.org/wiki/Web_scraping#HTML_parsing) with HTTP programming. Given the high costs of downloading large amounts of proprietary data, web scraping is sometimes the only option for small scale research projects. Google maps for example, the defacto mapping and navigation platform (as of 2020/2021), has a monopoly on information about the location of businesses across the globe. To get access to this information is very very costly (problems of monoply rule), therefore scraping the data is one of the only options for students. This tutorial demonstrates how to scrape Google Maps POI (points of interest) data in NYC with Python using the Selenium and Beautiful Soup libraries. 

The first step is to import all the necessary libraries:
- [Requests](https://requests.readthedocs.io/en/master/): a HTTP library for Python
- [Selenium](https://www.selenium.dev/selenium/docs/api/py/): a tool for automating web browsing
- [BeautfiulSoup](https://en.wikipedia.org/wiki/Beautiful_Soup_(HTML_parser)): a package to parse HTML elements
- [Shapely](https://shapely.readthedocs.io/en/stable/manual.html): a library for manipulaitng and analysing geometry
- [Geopandas](https://geopandas.org/index.html): similar to pandas with extensions to make it easier to work  with geospatial data.

In [1]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import geopandas as gpd
import time
import re
from shapely.geometry import Point

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

### Search requests

Every request to google maps usually starts from a location and a search query. This first section involves gathering a list of locations (latitudes & longitudes) and search queries (types of POI) to feed into the base URL below. The base url is the request we send to Google and what we'll get back is a selection of POI (businesses) for that location related to the search category.

In [3]:
base_url = "https://www.google.com/maps/search/{0}/@{1},{2},16z"

### Search locations

Any search in Google maps requires a location to search from, typically a latitude and longitude pair. Below we use the centroids of select zipcodes within NYC to use as our search locations. There are only 11 zip codes below but this could easily be scaled to the entire city or country.

In [6]:
conn = pg.connect(
    host=os.environ['aws_db_host'],
    port="5432",
    user=os.environ['aws_db_u'],
    password=os.environ['aws_db_p']
)

In [7]:
gdf = gpd.read_postgis('''
    SELECT 
        region_id,
        geom AS geom
    FROM geographies.zipcodes
    WHERE region_id IN (
    '10001', '10002', '10003', '10004', 
    '10005', '10006', '10007', '10008', 
    '10009', '10010', '10011'
    )
    ''', conn)

In [8]:
coords = gdf['geom'].centroid

In [10]:
xs = coords.apply(lambda x: x.x).tolist()
ys = coords.apply(lambda x: x.y).tolist()

<br>

Here we outline the search categories.

In [11]:
searches = ['Restaurants', 'bakery', 'coffee', 'gym', 'yoga', 
            'school', 'clothing', 'electronics', 
            'beauty', 'hardware', 'daycare', 'galleries',
            'museums', 'Hotels', 'deli', 
            'liquor', 'bar', 'Groceries', 'Takeout', 
            'Banks', 'Pharmacies']

### Sending a GET request

The next step is to send a request to Google's servers about the information we would like returned. This type of request is called a [GET request](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods) in HTTP programming. 

Below we'll send a single test request to the servers to see what information we get back. We insert a single lat/lng pair into the **base_url** variable. 

In [13]:
url = base_url.format(searches[0], xs[0], ys[0])

<br>

The next portion of code starts a selenium web driver (the vehicle that powers automated web browsing) and specifies a few browsing options. Specifically we state the web browser should run *headless*, meaning it should not open up a new browser window, and install a Chrome browser.

In [15]:
delay = 10
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), 
                           options=chrome_options)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


<br>

### Intentionally slowing down

Although web scraping is not [illegal](https://twitter.com/OrinKerr/status/1171116153948626944), researchers should be careful when scraping private websites. Not due to legality issues, but due to being blocked by the company you're trying to scrape from. Web scraping tools like Selenium browse pages in light speed, way faster than any human, therefore it is easy for company's servers to spot when individuals are trying to scrape their sites. This next section will introduce a few options to intentionally slow down our browsing. Specifically we tell the driver object to wait until certain conditions have been met before moving forward. Once conditions have been met we download the HTML information returned by Google.

In [16]:
driver.get(url)
try:
    # wait for button to be enabled
    WebDriverWait(driver, delay).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'section-result'))
    )
    driver.implicitly_wait(40)
    html = driver.find_element_by_tag_name('html').get_attribute('innerHTML')
except TimeoutException:
    print('Loading took too much time!')
else:
    print('getting html')
    html = driver.page_source
finally:
    driver.quit()

getting html


<br>

Now that we have the HTML data, we feed this into Beautiful Soup for parsing, without having to worry about tripping up Google's servers.

In [17]:
soup = BeautifulSoup(html)
results = soup.find_all('a', {'class': 'section-result'})

<br> 

Let's view one item from the results returned

In [22]:
results[0]

<a aria-label="Breads Bakery" class="section-result" data-result-index="1" data-section-id="ZXf8if7__or:1" jsaction="pane.resultSection.click;keydown:pane.resultSection.keydown;mouseover:pane.resultSection.in;mouseout:pane.resultSection.out;focus:pane.resultSection.focusin;blur:pane.resultSection.focusout" jsan="t-nqWj9GyoaSo,7.section-result,0.aria-label,0.data-result-index,0.data-section-id,0.role,0.ved,0.tabindex,22.jsaction" jstcache="375" role="button" tabindex="0" ved="1i:4,t:12690,e:3,r:1,p:XoTyX72tFa_X5gLZgYS4Aw:1"> <div class="section-ad-result-ad-banner" jstcache="123"></div> <div class="section-result-content"> <div class="section-result-text-content" jsan="7.section-result-text-content" jstcache="124"> <div class="section-result-header-container"> <div class="section-result-header" jsan="7.section-result-header" jstcache="125"> <div class="section-result-partial-result-justification" jstcache="126" style="display:none"> <span jstcache="127" style="display:none"> "<span jstc

<br>
<br>

As you can see, there is a ton of information returned but it is not in a human readable format.

To interpret this information we will have to go to a page from google maps and understand what our information is represented as in [HTML format](https://www.w3schools.com/whatis/whatis_htmldom.asp).

<img src="../static/img/web_elements.png">

<br>
<br>

Above is an example of a restaurant in Brooklyn as seen from Google maps, alongside the HTML representation. To open up the pop-up, right click anywhere on a page in Google maps and click **inspect elements**. Now hovering over a certain piece of information will tell you what HTML tag, class or ID the element has been assigned. For example, the name of the restaurant above, Park Plaza, is in a **H3** tag inside another **span** tag (highlighted in bliue at the bottom). Now we know the HTML representation, we can more easily parse the data with Beautiful Soup to return the information we want.

Below we ask Beatiful Soup to find a H3 tag element and then get the text from the subsequent span element.

In [27]:
results[0].find('h3').span.text

'Breads Bakery'

<br>

We can now scale the above logic to get all the attributes we're interested in (name, address, price, tags, etc.), from all the zip codes, and all the POI categories. We'll wrap the POI parsing in a nested loop and save the results to a CSV using Pandas. We also add Python's built-in **try** and **except** objects to the HTML parsing logic so that the loop continues even when there data is not returned as expected (a common thing in web scraping). At the end of the loop we will delay the next loop by 15 seconds to not trigger Google's servers.

In [89]:
for search in searches:   
    for coord in zip(ys, xs):
        filename = search + str(coord[0]).replace('.', '') + '_' + str(coord[1]).replace('.', '') + '.csv'
        if not os.path.isfile('data/'+filename):
            url = base_url.format(search, coord[0], coord[1])
            delay = 10
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            driver = webdriver.Chrome(ChromeDriverManager().install(), 
                                       options=chrome_options)
            driver.get(url)

            try:
                # wait for button to be enabled
                WebDriverWait(driver, delay).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'section-result'))
                )
                driver.implicitly_wait(40)
                html = driver.find_element_by_tag_name('html').get_attribute('innerHTML')
            except TimeoutException:
                print('Loading took too much time!')
            else:
                print('getting html')
                html = driver.page_source
            finally:
                driver.quit()
            soup = BeautifulSoup(html)
            results = soup.find_all('a', {'class': 'section-result'})
            data = []
            for item in results:
                try:
                    name = item.find('h3').span.text
                except:
                    name = None
                try:
                    rating = item.find('span', {'class': 'cards-rating-score'}).text
                except:
                    rating = None
                try:
                    num_reviews = item.find('span', {'class': 'section-result-num-ratings'}).text
                except:
                    num_reviews = None
                try:
                    cost = item.find('span', {'class': 'section-result-cost'}).text
                except:
                    cost = None
                try:
                    tags = item.find('span', {'class': 'section-result-details'}).text
                except:
                    tags = None
                try:
                    addr = item.find('span', {'class': 'section-result-location'}).text
                except:
                    addr = None
                try:
                    descrip = item.find('div', {'class': 'section-result-description'}).text
                except:
                    descrip = None
                data.append({'name': name, 'rating': rating, 'num_revs': num_reviews,
                             'cost': cost, 'tags': tags, 'address': addr, 
                             'description': descrip})
            temp_df = pd.DataFrame(data)
            temp_df.to_csv('data/' + filename)
            time.sleep(15)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
getting html


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
getting html


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


...

 
getting html


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
getting html


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/carlo/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


 
getting html


End!

<br>
<br>
<br>

(optional) code to call Google Places API to get more specifics

In [28]:
# addresses = []
# lats = []
# lngs = []
# for idx, address in enumerate(combined['address']):
#     try:
#         name = combined['name'].iloc[idx]
#         search = name + ', ' + address
#         api_key = os.environ['google_places_key']
#         url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={0}&inputtype=textquery&fields=formatted_address,geometry&key={1}".format(search, api_key)
#         r = requests.get(url)
#         features = r.json()
#         addresses.append(features['candidates'][0]['formatted_address'])
#         lats.append(features['candidates'][0]['geometry']['location']['lat'])
#         lngs.append(features['candidates'][0]['geometry']['location']['lng'])
#     except:
#         addresses.append(None)
#         lats.append(None)
#         lngs.append(None)

In [29]:
# combined['lat'] = lats
# combined['lng'] = lngs
# combined['formatted_address'] = addresses